# Homework-1 Question-4

## Alloy Bending
The company Steelco has received an order for 500 tons of steel to be used
in shipbuilding. The steel must have the following characteristics:

Chemical Element | Minimum Grade (%) | Maximum Grade (%)
--- | --- | --- 
Carbon (C) | 2 | 3
Copper (Cu) | 0.4 | 0.6
Manganese (Mn) | 1.2 | 1.65

The company has seven different raw materials in stock that may be used for the production of this
steel. The following table lists the grades, available amounts and prices for all materials:

Raw Material | C%  | Cu% | Mn% | Availability in tons | Cost in $/ton
-------------| --- | ----|-----|------|----
Iron alloy 1 | 2.5 |     | 1.3 | 400  | 200
Iron alloy 2 | 3   |     | 0.8 | 300  | 250
Iron alloy 3 |     | 0.3 |     | 600  | 150
Copper 1     |     | 90  |     | 500  | 220
Copper 2     |     | 96  | 4   | 200  | 240
Aluminum 1   |     | 0.4 | 1.2 | 300  | 200
Aluminum 2   |     | 0.6 |     | 250  | 165

## Problem Data

In [1]:
# raw materials list, Iron Alloy1,2,3, Copper 1,2, Aluminium 1,2
raw_materials = [:IronAlloy1, :IronAlloy2, :IronAlloy3, :Copper1, :Copper2, :Aluminium1, :Aluminium2]

# elements in the steel
elements = [:Carbon, :Copper, :Manganese]

# cost of each raw material
cost = Dict(zip(raw_materials, [200, 250, 150, 220, 240, 200, 165]))

# availability of each raw material
availability = Dict(zip(raw_materials, [400, 300, 600, 500, 200, 300, 250]))

# weight of elements as per 500 tons of final steel, respecting min and max grades required
min_grade = Dict(zip(elements, [0.02, 0.004, 0.012].*500))
max_grade = Dict(zip(elements, [0.03, 0.006, 0.0165].*500))

# (raw_materials, elements)
using NamedArrays
# Composition matrix raw_materials vs elements
composition_mat = [0.025 0      0.013
                   0.03  0      0.008
                   0     0.003  0
                   0     0.9    0
                   0     0.96   0.04
                   0     0.004  0.012
                   0     0.006  0]

composition = NamedArray( composition_mat, (raw_materials,elements), ("raw_materials","elements") )
println(composition)

7×3 Named Array{Float64,2}
raw_materials ╲ elements │    Carbon     Copper  Manganese
─────────────────────────┼────────────────────────────────
IronAlloy1               │     0.025        0.0      0.013
IronAlloy2               │      0.03        0.0      0.008
IronAlloy3               │       0.0      0.003        0.0
Copper1                  │       0.0        0.9        0.0
Copper2                  │       0.0       0.96       0.04
Aluminium1               │       0.0      0.004      0.012
Aluminium2               │       0.0      0.006        0.0


## Problem Model

In [3]:
using JuMP

m = Model()

#Vectored variable to store quantities of all raw materials
@variable(m, quantities[raw_materials] >=0)

#Expressions for total cost of steel created and total quantity after mixing chosen raw materials
@expression(m, total_cost, sum(cost[r]*quantities[r] for r in raw_materials))
@expression(m, total_quantity, sum(quantities[r] for r in raw_materials))

@constraint(m, total_quantity == 500)  # Total quantity should meet the demand of 500 tons
                        
# Constraint on quantities of raw materials to be less than the available raw materials
@constraint(m, avail_constr[r in raw_materials], quantities[r] <= availability[r])
                        
# Constraint on min and max grade of elements required in the final 500 ton steel
@constraint(m, min_grade_constr[e in elements],
               sum(composition[r,e]*quantities[r] for r in raw_materials)
               >= min_grade[e] )
@constraint(m, max_grade_constr[e in elements],
               sum(composition[r,e]*quantities[r] for r in raw_materials)
               <= max_grade[e] )
        
@objective(m, Min, total_cost)  # Objective to minimize the final cost
solve(m)
println("Minimum Cost: \$", getobjectivevalue(m))
println(getvalue(quantities))
println(m)

Minimum Cost: $98121.63579168124
quantities: 1 dimensions:
[IronAlloy1] = 400.0
[IronAlloy2] = 0.0
[IronAlloy3] = 39.77630199231039
[   Copper1] = 0.0
[   Copper2] = 2.761272282418735
[Aluminium1] = 57.462425725270876
[Aluminium2] = 0.0

Min 200 quantities[IronAlloy1] + 250 quantities[IronAlloy2] + 150 quantities[IronAlloy3] + 220 quantities[Copper1] + 240 quantities[Copper2] + 200 quantities[Aluminium1] + 165 quantities[Aluminium2]
Subject to
 quantities[IronAlloy1] + quantities[IronAlloy2] + quantities[IronAlloy3] + quantities[Copper1] + quantities[Copper2] + quantities[Aluminium1] + quantities[Aluminium2] = 500
 quantities[IronAlloy1] ≤ 400
 quantities[IronAlloy2] ≤ 300
 quantities[IronAlloy3] ≤ 600
 quantities[Copper1] ≤ 500
 quantities[Copper2] ≤ 200
 quantities[Aluminium1] ≤ 300
 quantities[Aluminium2] ≤ 250
 0.025 quantities[IronAlloy1] + 0.03 quantities[IronAlloy2] ≥ 10
 0.003 quantities[IronAlloy3] + 0.9 quantities[Copper1] + 0.96 quantities[Copper2] + 0.004 quantities[Alumini

The minimum cost found by the model comes out to be \$98121.63. The final quantities used of all alloys is as mentioned in the output of previous cell.